In [ ]:
# Trainer로 MLM 훈련시킨 모델에 대해 평가해보는 예시 

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, DistilBertForMaskedLM,BertForMaskedLM, RobertaForMaskedLM
from transformers import AdamW, get_linear_schedule_with_warmup

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# eval 말뭉치 
is_Localfile = False   # 로컬파일이면 = True, 허깅페이스면 False
#eval_corpus = "bongsoo/bongeval"
eval_corpus = "bongsoo/moco_eval"
#eval_corpus = "../../data11/my_corpus/klue_dp_valid_text.txt"

# 기존 사전훈련된 모델
# model 타입 : 0=distilbert, 1=bert, 2=Roberta, 3=Kobert
#=>Roberta 모델에는 distilbert처럼 token_type_id 입력 없음.
model_type = 3
#model_path = "../../data11/model/distilbert/scratch/mdistilbert-scratchV-20/"
model_path = "skt/kobert-base-v1"

# 출력
#OUTPATH = '../../data11/model/distilbert/scratch/mdistilbert-scratchV-20-out/'
OUTPATH = './test'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size
token_max_len = 128   # token_seq_len
############################################################################

device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="distilbert-MLM-Test-Trainer", logfilename="../../log/distilbert-MLM-Test-Trainer")

In [ ]:
# tokeinzier 생성
if model_type == 3:
    from kobert_tokenizer import KoBERTTokenizer
    tokenizer = KoBERTTokenizer.from_pretrained(model_path)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_path, max_len=token_max_len, do_lower_case=False)

# fast 토크너나이즈인지 확인
print(f'{model_path} is_fast:{tokenizer.is_fast}')

# 모델 로딩 further pre-training 
if model_type == 0:
    model = DistilBertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
elif model_type == 1 or model_type == 3:
    model = BertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
elif model_type == 2:
    model = RobertaForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 

model.to(device)
model.eval() # 평가 

In [ ]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

if is_Localfile == False:
    # 평가 말뭉치 로딩
    eval_dataset1 = load_dataset(eval_corpus)
    eval_dataset = eval_dataset1['test']
else:
    # 파일 평가 말뭉치 로딩
    eval_dataset1 = load_dataset("text", data_files=eval_corpus)
    eval_dataset = eval_dataset1['train']

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset[0:3])

In [ ]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

if tokenizer.is_fast == True:
    %time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)
else:
    eval_dataset_fast = tokenizer(eval_dataset['text'], return_tensors='pt', max_length=token_max_len, truncation=True, padding='max_length')

In [ ]:
if tokenizer.is_fast == True:
    print(eval_dataset_fast)
    print(f'*fast_len:{len(eval_dataset_fast["text"])}, len:{len(eval_dataset["text"])}')  # fast_dataset과 dataset 길이를 비교함
    print(eval_dataset_fast['text'][0:2])
else:
    print(eval_dataset_fast)
    print(f'*fast_len:{len(eval_dataset_fast["input_ids"])}, len:{len(eval_dataset["text"])}')  # fast_dataset과 dataset 길이를 비교함
    print(eval_dataset_fast['input_ids'][0:2])

In [ ]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

if tokenizer.is_fast == True:
    # input_ids MLM 만들고 출력 해봄
    mlm_eval_sample = data_collator(eval_dataset_fast['input_ids'][0:2])

    print(f"eval_dataset_fast(MLM)=======================================")
    print(mlm_eval_sample['input_ids'][0])
    print(eval_dataset_fast['text'][0])

In [ ]:
# 테스트 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#========================================================================================
# 아래 computer_metrics 해도 안해도 동일한 loss 얻어옴
# => trainer = Trainer(compute_metrics=compute_metrics) 해줘야 함.

from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}
#========================================================================================

#########################################################################################
# hyper parameter 설정
#########################################################################################

eval_steps=int(len(eval_dataset_fast["input_ids"]))           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)

print(f'*total_optim_steps: {len(eval_dataset_fast["input_ids"])}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                       # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    seed=333,
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    #prediction_loss_only=True,
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
eval_dataset_fast_input_ids = eval_dataset_fast['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    eval_dataset=eval_dataset_fast_input_ids,      # 평가 데이터셋
    #compute_metrics=compute_metrics,
)


In [ ]:
# 평가 
metrics = trainer.evaluate()
logger.info('{}'.format(metrics))
            